<a href="https://colab.research.google.com/github/joomj2000/Reinforcement-Learning/blob/main/custom_gym_shooting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
!cp -r /content/drive/MyDrive/SKT_강화학습/gym_examples /content

In [ ]:
!ls gym_examples

envs  __init__.py


In [ ]:
!pip install gymnasium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 8.5 MB/s eta 0:00:00


In [ ]:
import gymnasium as gym
env = gym.make('gym_examples:gym_examples/ShootingAirplane-v0', render_mode='text')

In [ ]:
obs, info = env.reset()

In [ ]:
env.render()

         |         
         |         
         |         
         |      H  
         |    H H  
         |    HHHH 
         |    H H  
         |      H  



In [ ]:
obs, reward, done,_, info = env.step((0, 0))

In [ ]:
env.render()

M        |         
         |         
         |         
         |      H  
         |    H H  
         |    HHHH 
         |    H H  
         |      H  



In [ ]:
obs, reward, done, _,info = env.step((5, 5))

In [ ]:
env.render()

         |         
         |         
         |   H     
         | HHHHH   
         |   H     
    MM   |  HHH    
         |         
    M    |         



In [ ]:
obs, reward, done,_, info = env.step((3, 3))
env.render()

         |         
         |         
         |   H     
   H     | HHHHH   
         |   H     
    MM   |  HHH    
         |         
    M    |         



In [ ]:
reward

1

In [ ]:
obs, reward, done,_,info = env.step((5, 4))
env.render()

         |         
         |         
         |   H     
   H     | HHHHH   
         |   H     
    MM   |  HHH    
         |         
    M    |         



In [ ]:
reward

-1

# DQN

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as T

In [ ]:
gamma = 0.99
epsilon = 1.0
epsilon_max = 1.0
epsilon_min = 0.1
epsilon_interval = epsilon_max - epsilon_min
batch_size = 16
max_steps_per_episode = 60
max_episodes = 10000

In [ ]:
env.observation_space

Box(0, 255, (8, 8, 1), uint8)

In [ ]:
env.action_space

MultiDiscrete([8 8])

In [ ]:
num_actions = 64

In [ ]:
class QModel(nn.Module):
  def __init__(self, num_actions):
    super(QModel, self).__init__()
    self.dropout = nn.Dropout(p=0.3)
    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding='same')
    self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding='same')
    self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=1)
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(1152, 512)
    self.fc2 = nn.Linear(512, num_actions)

  def forward(self, x):
    x = nn.functional.relu(self.conv1(x))
    x = nn.functional.relu(self.conv2(x))
    x = self.dropout(x)
    x = nn.functional.relu(self.conv3(x))
    x = self.flatten(x)
    x = nn.functional.relu(self.fc1(x))
    x = self.dropout(x)
    return self.fc2(x)

In [ ]:
model = QModel(num_actions)

In [ ]:
model_target = QModel(num_actions)

In [ ]:
loss_function = nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00025)

In [ ]:
action_history = []
action_mask_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []

In [ ]:
episode_reward_history = []
running_reward = 0.
episode_count = 0
frame_count = 0

In [ ]:
epsilon_random_frames = 50000
# Number of frames for exploration
epsilon_greedy_frames = 200000.0
# Maximum replay length
# Note: The Deepmind paper suggests 1000000 however this causes memory issues
max_memory_length = 500000
# Train the model after 4 actions
update_after_actions = 4
# How often to update the target network
update_target_network = 10000

In [ ]:
def preprocess_state(obs):
  st = torch.from_numpy(obs).squeeze()
  st = st.to(torch.int64)
  st = torch.nn.functional.one_hot(st, num_classes=3)
  st = st.permute(2, 0, 1)
  return st.to(torch.float32)

In [ ]:
board, info = env.reset()

In [ ]:
board.shape

(8, 8, 1)

In [ ]:
st = preprocess_state(board)
st.shape

torch.Size([3, 8, 8])

In [ ]:
st

tensor([[[1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
       

In [ ]:
def get_greedy_epsilon(model, state, mask):
  global epsilon

  #if frame_count < epsilon_random_frames or np.random.rand(1)[0] < epsilon:
  if np.random.rand(1)[0] < epsilon:
    action = np.random.choice([ i for i in range(num_actions) if mask[i] == 1 ])
  else:
    with torch.no_grad():
      # add a batch axis
      state_tensor = state.unsqueeze(0)
      # compute the q-values
      q_values = model(state_tensor)
      # select the q-values of valid actions
      action = torch.argmax(
        q_values.squeeze() + torch.from_numpy(mask) * 100., dim=0)

  epsilon -= epsilon_interval / epsilon_greedy_frames
  epsilon = max(epsilon, epsilon_min)
  return action

In [ ]:
def get_greedy_action(model, state, mask):
  global epsilon
  with torch.no_grad():
    state_tensor = state.unsqueeze(0) # batch dimension
    q_values = model(state_tensor)
    action = torch.argmax(
      q_values.squeeze() + torch.from_numpy(mask) * 100.,dim=0)
  return action

In [ ]:
def sample_batch(_batch_size):
  indices = np.random.choice(range(len(done_history)), size=_batch_size, replace=False)
  state_sample = np.array([state_history[i].squeeze(0).numpy() for i in indices])
  state_next_sample = np.array([state_next_history[i].squeeze(0).numpy() for i in indices])
  rewards_sample = np.array([rewards_history[i] for i in indices], dtype=np.float32)
  action_sample = np.array([action_history[i] for i in indices])

  # action mask is the mask for the valid actions at the '''next''' state
  action_mask_sample = np.array([action_mask_history[i] for i in indices])
  done_sample = np.array([float(done_history[i]) for i in indices])
  return state_sample, state_next_sample, rewards_sample, action_sample, action_mask_sample, done_sample

In [ ]:
def update_network():
  state_sample, state_next_sample, rewards_sample, action_sample, action_mask_sample, done_sample = \
    sample_batch(batch_size)

  state_sample = torch.tensor(state_sample, dtype=torch.float32)
  state_next_sample = torch.tensor(state_next_sample, dtype=torch.float32)
  action_sample = torch.tensor(action_sample, dtype=torch.int64)
  action_mask_sample = torch.tensor(action_mask_sample, dtype=torch.int64)
  rewards_sample = torch.tensor(rewards_sample, dtype=torch.float32)
  done_sample = torch.tensor(done_sample, dtype=torch.float32)

  with torch.no_grad():
    future_rewards = model_target(state_next_sample)
    max_q_values = torch.max (
        future_rewards + action_mask_sample * 100.,
    dim=1).values.detach() - 100.
    target_q_values = rewards_sample + gamma * max_q_values * (1. - done_sample)

  q_values = model(state_sample)
  q_values_action = q_values.gather(dim=1, index=action_sample.unsqueeze(1)).squeeze(1)
  loss = loss_function(q_values_action, target_q_values)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()


In [89]:
for _ in range(max_episodes):
  state, info = env.reset()
  state = preprocess_state(state)
  action_mask = info['action_mask'].reshape((-1,))
  episode_reward = 0

  for timestep in range(1, max_steps_per_episode):
    frame_count += 1

    action = get_greedy_epsilon(model, state, action_mask)

    state_next, reward, done,_, info = env.step((action // 8, action % 8))
    state_next = preprocess_state(state_next)
    action_mask = info['action_mask'].reshape((-1,))

    episode_reward += reward

    action_history.append(action)
    action_mask_history.append(action_mask)
    state_history.append(state)
    state_next_history.append(state_next)
    rewards_history.append(reward)
    done_history.append(done)

    state = state_next

    if frame_count % update_after_actions == 0 and len(done_history) > batch_size:
      update_network()

    if frame_count % update_target_network == 0:
      model_target.load_state_dict(model.state_dict())

    if len(rewards_history) > max_memory_length:
      del rewards_history[:1]
      del state_history[:1]
      del state_next_history[:1]
      del action_history[:1]
      del action_mask_history[:1]
      del done_history[:1]
    if done:
      break

  episode_count +=1
  episode_reward_history.append(episode_reward)

  if len(episode_reward_history)> 100:
    del episode_reward_history[0]

  running_reward = np.mean(episode_reward_history)

  if episode_count % 10 == 0:
    print(episode_count, frame_count, running_reward)



10 580 -39.5
20 1160 -39.65
30 1742 -39.7
40 2294 -38.775
50 2863 -38.68
60 3420 -38.35
70 4002 -38.642857142857146
80 4570 -38.5625
90 5144 -38.56666666666667
100 5691 -38.26
110 6265 -38.17
120 6843 -38.09
130 7418 -37.94
140 7987 -38.07
150 8568 -38.27
160 9135 -38.41
170 9711 -38.25
180 10286 -38.36
190 10818 -37.84
200 11362 -37.77
210 11928 -37.67
220 12475 -37.3
230 13042 -37.26
240 13569 -36.86
250 14121 -36.43
260 14680 -36.31
270 15226 -35.93
280 15777 -35.67
290 16314 -35.8
300 16887 -36.17
310 17439 -36.05
320 17946 -35.59
330 18506 -35.46
340 18994 -35.05
350 19494 -34.45
360 19997 -33.83
370 20535 -33.75
380 21043 -33.22
390 21594 -33.32
400 22087 -32.42
410 22605 -32.0
420 23145 -32.41
430 23662 -31.92
440 24184 -32.26
450 24671 -32.13
460 25148 -31.85
470 25662 -31.63
480 26135 -31.26
490 26631 -30.65
500 27144 -30.89
510 27652 -30.79
520 28177 -30.64
530 28664 -30.36
540 29183 -30.29
550 29685 -30.48
560 30153 -30.37
570 30664 -30.32
580 31159 -30.54
590 31641 -30.42
6

In [90]:
import time, sys
from IPython.display import clear_output
board, info = env.reset()
state = preprocess_state(board)
action_mask = info['action_mask'].reshape((-1,))
done = False
env.render()
while not done:
  action = get_greedy_action(model, state, action_mask)
  print("action: ({}, {})".format(action // 8, action % 8))
  sys.stdout.flush()
  time.sleep(1.0)
  clear_output(wait=False)
  board, reward, done, _,info = env.step((action // 8, action % 8))
  state = preprocess_state(board)
  action_mask = info['action_mask'].reshape((-1,))
  env.render()


         |         
  MHM    |    H    
 HMH     |  H H    
 HHHHM   |  HHHH   
 H H M   |  H H    
 M HM    |    H    
         |         
         |         

